# Dask Array Support to AnnData

## Initalizing

First let's do our imports and initalize adata objects with the help of the `adata_with_dask` function defined below.

In [1]:
import dask.array as da
import numpy as np
import pandas as pd
import anndata as ad

In [2]:
def adata_with_dask(M, N):
    adata_dict = {}
    adata_dict["X"] = da.random.random((M, N))
    adata_dict["dtype"] = np.float64
    adata_dict["obsm"] = dict(
        a=da.random.random((M, 100)),
    )
    adata_dict["layers"] = dict(
        a=da.random.random((M, N)),
    )
    adata_dict["obs"] = pd.DataFrame(
        {"batch": np.random.choice(["a", "b"], M)},
        index=[f"cell{i:03d}" for i in range(M)],
    )
    adata_dict["var"] = pd.DataFrame(index=[f"gene{i:03d}" for i in range(N)])
    
    
    return ad.AnnData(**adata_dict)

Here is how our adata looks like

In [3]:
adata = adata_with_dask(10000,10000)
adata

AnnData object with n_obs × n_vars = 10000 × 10000
    obs: 'batch'
    obsm: 'a'
    layers: 'a'

## Representation of Dask Arrays

Dask arrays consists of chunks that can be distributed in clusters. In the figure below, each small square represents a chunk that form a dask array. In principle these some of these chunks could be in different machines (clusters).

In [4]:
adata.X

dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(2500, 2500), chunktype=numpy.ndarray>

In [5]:
adata.obsm['a']

dask.array<random_sample, shape=(10000, 100), dtype=float64, chunksize=(10000, 100), chunktype=numpy.ndarray>

## The Computation Graph
The graph layer in the `Count` row refers to the layer of the computation graph of the chunks, i.e. which operations are applied to them. We have this because the operations done on Dask arrays aren't computed instantly. This way, Dask array can optimize the queries we issued to it. It also won't keep our resources occupied for the results we expect later. Below is a representation of the chunks we initially created.

In [6]:
adata.X.visualize()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

We now show an example for this computation graph on dask arrays to understand it better. This part is not technically relevant to AnnData.

In [7]:
xsum = adata.X.sum(axis=1) # do a sum on axis=1
xsum

dask.array<sum-aggregate, shape=(10000,), dtype=float64, chunksize=(2500,), chunktype=numpy.ndarray>

Note that this computation isn't necessarily done yet, but rather saved to actually run it later.   If we investigate the computation graph of this result, we can see that for this operation some chunks aren't dependent on each other. This might give a hint to the Dask framework to store the chunks that depend on each other to the same cluster. For this simple exercise, all the chunks can be stored in one machine, but when it is impossible to store all the four chunks into one machine this will come in handy.

In [8]:
xsum.visualize()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

But coming back to our anndata tutorial we will see that nothing changed in our adata.

In [9]:
adata.X.visualize()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

## Concatenation

In this section we will cover how concatenation on anndata objects that use Dask arrays looks like. We first create another anndata object to concatenate with.

In [10]:
adata2 = adata_with_dask(10000,10000)
adata2

AnnData object with n_obs × n_vars = 10000 × 10000
    obs: 'batch'
    obsm: 'a'
    layers: 'a'

We can see that the X attribute of adata also consist of four chunks.

In [11]:
adata2.X

dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(2500, 2500), chunktype=numpy.ndarray>

In [12]:
adata_concat = ad.concat([adata,adata2],index_unique='id')

When we concatenate the whole object you can see that in the X of the result consists of eight chunks and they quite probably are just the source chunks aligned.

In [13]:
adata_concat.X

dask.array<concatenate, shape=(20000, 10000), dtype=float64, chunksize=(2500, 2500), chunktype=numpy.ndarray>

To confirm this we look at the computation graph. We can confirm that this new object's X is just the chunks of source X's put together.

In [14]:
adata_concat.X.visualize()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

Here for the obsm we also this. The chunk from both are just stacked on top.

In [15]:
adata_concat.obsm['a']

dask.array<concatenate, shape=(20000, 100), dtype=float64, chunksize=(10000, 100), chunktype=numpy.ndarray>

In [16]:
adata_concat.obsm['a'].visualize()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

## Views

Let's see how our views of anndata objects play with Dask arrays.

### Slice View



We take a slice of the concatenated adatas. This operation returns a view of the adata which means that the resulting adata holds a view of the source adata's Dask array, namely the `DaskArrayView` class,  which is a completely different object than Dask array.

In [17]:
adata_slice_view = adata_concat[:500, :][:, :500]

Below, you can see the values of the X attribute of the result. Which implies that the resulting object isn't "lazy" like Dask arrays. 

In [18]:
adata_slice_view.X, adata_slice_view.shape

(DaskArrayView([[0.81944854, 0.57749801, 0.96662254, ..., 0.32336001,
                 0.5268472 , 0.58104636],
                [0.97506059, 0.54932614, 0.62854065, ..., 0.61735101,
                 0.46933571, 0.94360259],
                [0.42689006, 0.54068094, 0.27245775, ..., 0.08559342,
                 0.71470039, 0.85156446],
                ...,
                [0.04739288, 0.84608847, 0.25487207, ..., 0.01877132,
                 0.49316309, 0.19942156],
                [0.82151726, 0.42624912, 0.827517  , ..., 0.69564292,
                 0.06624968, 0.19592569],
                [0.33542202, 0.13316215, 0.37816198, ..., 0.17696379,
                 0.40050429, 0.56989527]]),
 (500, 500))

But our original adata remains unchanged.

In [19]:
adata_concat.X

dask.array<concatenate, shape=(20000, 10000), dtype=float64, chunksize=(2500, 2500), chunktype=numpy.ndarray>

### Index List View

In [20]:
small_view = adata_concat[[12,12,3,5,53],[1,2,5]]
small_view

View of AnnData object with n_obs × n_vars = 5 × 3
    obs: 'batch'
    obsm: 'a'
    layers: 'a'

In [21]:
small_view.X

DaskArrayView([[0.99162464, 0.215163  , 0.45043157],
               [0.99162464, 0.215163  , 0.45043157],
               [0.69536295, 0.48615478, 0.93954781],
               [0.34846512, 0.95620265, 0.6488894 ],
               [0.79371857, 0.89436834, 0.18337423]])

### View by Category

In [22]:
categ_view = adata_concat[adata_concat.obs['batch'] == 'b']

In [23]:
categ_view.X

DaskArrayView([[0.81944854, 0.57749801, 0.96662254, ..., 0.29731019,
                0.40350271, 0.09477139],
               [0.97506059, 0.54932614, 0.62854065, ..., 0.05971807,
                0.94450394, 0.7031698 ],
               [0.42689006, 0.54068094, 0.27245775, ..., 0.27021215,
                0.60209725, 0.49595075],
               ...,
               [0.24356409, 0.92550514, 0.48320369, ..., 0.01056079,
                0.75637283, 0.18503286],
               [0.83408785, 0.62531777, 0.51587336, ..., 0.05780624,
                0.43723214, 0.82418455],
               [0.60928596, 0.63411101, 0.8084821 , ..., 0.84715743,
                0.08837802, 0.01680197]])

## To Memory

In [24]:
adata_concat.X

dask.array<concatenate, shape=(20000, 10000), dtype=float64, chunksize=(2500, 2500), chunktype=numpy.ndarray>

Here no copies are made, only the result of the lazy object is asked to be materialized.

In [25]:
adata_mem = adata_concat.to_memory(copy=False)
adata_mem.X

array([[0.81944854, 0.57749801, 0.96662254, ..., 0.29731019, 0.40350271,
        0.09477139],
       [0.97506059, 0.54932614, 0.62854065, ..., 0.05971807, 0.94450394,
        0.7031698 ],
       [0.42689006, 0.54068094, 0.27245775, ..., 0.27021215, 0.60209725,
        0.49595075],
       ...,
       [0.87687017, 0.07092441, 0.25932323, ..., 0.22925315, 0.92566352,
        0.8017111 ],
       [0.2319559 , 0.21290886, 0.92660466, ..., 0.17845157, 0.64034381,
        0.70860789],
       [0.04458435, 0.34683586, 0.53871565, ..., 0.81779622, 0.08099927,
        0.82453518]])

If you want to both materialize the result and copy.

In [26]:
del adata_mem

In [27]:
adata_mem = adata_concat.to_memory(copy=True)
adata_mem.X

array([[0.81944854, 0.57749801, 0.96662254, ..., 0.29731019, 0.40350271,
        0.09477139],
       [0.97506059, 0.54932614, 0.62854065, ..., 0.05971807, 0.94450394,
        0.7031698 ],
       [0.42689006, 0.54068094, 0.27245775, ..., 0.27021215, 0.60209725,
        0.49595075],
       ...,
       [0.87687017, 0.07092441, 0.25932323, ..., 0.22925315, 0.92566352,
        0.8017111 ],
       [0.2319559 , 0.21290886, 0.92660466, ..., 0.17845157, 0.64034381,
        0.70860789],
       [0.04458435, 0.34683586, 0.53871565, ..., 0.81779622, 0.08099927,
        0.82453518]])

# IO operations

Read/Write operations on `h5ad` and `Zarr` are supported. One should note that the lazy objects are materialized when this is called. For now, the anndata loaded from file won't be loaded with dask arrays in it.

## Write h5ad

In [28]:
adata = adata_with_dask(100,100)

In [29]:
adata.write_h5ad('a1.h5ad')

In [30]:
adata.X

dask.array<random_sample, shape=(100, 100), dtype=float64, chunksize=(100, 100), chunktype=numpy.ndarray>

## Read h5ad

In [31]:
h5ad_adata = ad.read_h5ad('a1.h5ad')

In [32]:
h5ad_adata.X

array([[0.23467525, 0.30740323, 0.74707737, ..., 0.73567116, 0.8075702 ,
        0.12136477],
       [0.92602808, 0.41365613, 0.1436757 , ..., 0.78506454, 0.39878838,
        0.32780577],
       [0.79105254, 0.07633488, 0.52624369, ..., 0.32672965, 0.18974894,
        0.06443115],
       ...,
       [0.86949841, 0.32182262, 0.85985404, ..., 0.53748529, 0.68370329,
        0.5003929 ],
       [0.12446353, 0.07439336, 0.16725495, ..., 0.60285862, 0.02256005,
        0.35275169],
       [0.88416568, 0.31795235, 0.7733777 , ..., 0.11549577, 0.24650333,
        0.05702779]])

## Write zarr

In [33]:
adata.write_zarr('a2.zarr')

In [34]:
adata.X

dask.array<random_sample, shape=(100, 100), dtype=float64, chunksize=(100, 100), chunktype=numpy.ndarray>

## Read zarr

In [35]:
zarr_adata = ad.read_zarr('a2.zarr')

In [36]:
zarr_adata.X

array([[0.23467525, 0.30740323, 0.74707737, ..., 0.73567116, 0.8075702 ,
        0.12136477],
       [0.92602808, 0.41365613, 0.1436757 , ..., 0.78506454, 0.39878838,
        0.32780577],
       [0.79105254, 0.07633488, 0.52624369, ..., 0.32672965, 0.18974894,
        0.06443115],
       ...,
       [0.86949841, 0.32182262, 0.85985404, ..., 0.53748529, 0.68370329,
        0.5003929 ],
       [0.12446353, 0.07439336, 0.16725495, ..., 0.60285862, 0.02256005,
        0.35275169],
       [0.88416568, 0.31795235, 0.7733777 , ..., 0.11549577, 0.24650333,
        0.05702779]])

Notice how they are loaded as arrays rather than dask arrays.

## Dask Array Support for Other Fields

This is the list of operations and in which fields they are supported, although some might have not been covered in this tutorial.

The following work with operations anndata supported before are also supported now with Dask arrays:
- anndata.concat()
- Views
- copy()
- to_memory() (changed behaviour)
- read/write on h5ad/zarr

**X, obsm, varm, obsp, varp, layers, uns, and raw** attributes are all supported and tested.

Note: scipy.sparse array wrapped with dask array doesn't play well. This is mainly because of the inconsistent numpy api support of scipy.sparse. Even though not explicitly tested, a sparse array that supports the numpy api should theoretically work well.